# Poutyne's Tips and Tricks

Poutyne also over a variety of tools for fine-tuning the information generated during the training, such as colouring the training update message, a progress bar, multi-GPUs, user callbacks interface and a user naming interface for the metrics' names. 

We will explore those tools using a different problem presented in [Introduction to Pytorch and Poutyne](https://github.com/GRAAL-Research/poutyne/blob/master/examples/introduction_pytorch_poutyne.ipynb).

Let's install the latest version of Poutyne and colorama (if they are not already), and import all the needed packages.

In [1]:
%pip install --upgrade fasttext
%pip install --upgrade poutyne
%pip install --upgrade colorama
%matplotlib inline
import contextlib
import os
import pickle
import re
import sys
from io import TextIOBase

import fasttext
import fasttext.util
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence
from torch.utils.data import DataLoader

from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, Callback, SKLearnMetrics

Also, we need to set Pythons's, NumPy's and PyTorch's seeds by using Poutyne function so that our training is (almost) reproducible.

In [2]:
set_seeds(42)

# Train a Recurrent Neural Network (RNN)

In this notebook, we train an RNN, or more precisely, an LSTM, to predict the sequence of tags associated with a given address, known as parsing address.

This task consists of detecting, by tagging, the different parts of an address such as the civic number, the street name or the postal code (or zip code). The following figure shows an example of such a tagging.

![address parsing canada](https://poutyne.org/_images/address_parsing.png)

Since addresses are written in a predetermined sequence, RNN is the best way to crack this problem. For our architecture, we will use two components, an RNN and a fully-connected layer.

## Training Constants

Now, let's set our training constants. We first have the CUDA device used for training if one is present. Secondly, we set the batch size (i.e. the number of elements to see before updating the model) and the learning rate for the optimizer.

In [3]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 32
lr = 0.1

## RNN

For the first components, instead of using a vanilla RNN, we use a variant of it, know as a long short-term memory (LSTM) (to learn more about [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). For now, we use a single layer unidirectional LSTM. 

Also, since our data is textual, we will use the well-known word embeddings to encode the textual information. So the LSTM input and hidden state dimensions will be of the same size. This size corresponds to the word embeddings dimension, which in our case will be the [French pre trained](https://fasttext.cc/docs/en/crawl-vectors.html) fastText embeddings of dimension 300. 

> See [here](https://discuss.pytorch.org/t/could-someone-explain-batch-first-true-in-lstm/15402) the explanation why we use the `batch_first` argument.

In [4]:
dimension = 300
num_layer = 1
bidirectional = False

lstm_network = nn.LSTM(input_size=dimension,
                       hidden_size=dimension,
                       num_layers=num_layer,
                       bidirectional=bidirectional,
                       batch_first=True)


## Fully-connected Layer

We use this layer to map the representation of the LSTM (300) to the tag space (8, the number of tags) and predict the most likely tag using a softmax.

In [5]:
input_dim = dimension #the output of the LSTM
tag_dimension = 8

fully_connected_network = nn.Linear(input_dim, tag_dimension)

## The Dataset

Now let's download our dataset; it already split into a train, valid and test set using the following.

In [6]:
def download_data(saving_dir, data_type):
    """
    Function to download the dataset using data_type to specify if we want the train, valid or test.
    """

    root_url = "https://graal-research.github.io/poutyne-external-assets/tips_and_tricks_assets/{}.p"

    url = root_url.format(data_type)
    r = requests.get(url)
    os.makedirs(saving_dir, exist_ok=True)

    open(os.path.join(saving_dir, f"{data_type}.p"), 'wb').write(r.content)
    
download_data('./data/', "train")
download_data('./data/', "valid")
download_data('./data/', "test")

Now let's load in memory the data.

In [7]:
# load the data

train_data = pickle.load(open("./data/train.p", "rb"))  # 80,000 examples
valid_data = pickle.load(open("./data/valid.p", "rb"))  # 20,000 examples
test_data = pickle.load(open("./data/test.p", "rb"))  # 30,000 examples

If we take a look at the training dataset, it's a list of 80,000 tuples where the first element is the full address, and the second element is a list of the tag (the ground truth).

In [8]:
train_data[0:2]

[('33 harnesworth crescent city of hamilton ontario l8b0j3',
  ['StreetNumber',
   'StreetName',
   'StreetName',
   'Municipality',
   'Municipality',
   'Municipality',
   'Province',
   'PostalCode']),
 ('1449 mouettes longueuil quebec j4j5k4',
  ['StreetNumber', 'StreetName', 'Municipality', 'Province', 'PostalCode'])]

Since the address is a text, we need to *convert* it into categorical value, such as word embeddings, for that we will use a vectorizer. This embedding vectorizer will be able to extract for every word embedding value.

In [9]:
# We use this class so that the download templating of the fasttext
# script be not buggy as hell in notebooks.
class LookForProgress(TextIOBase):
    def __init__(self, stdout):
        self.stdout = stdout
        self.regex = re.compile(r'([0-9]+(\.[0-9]+)?%)', re.IGNORECASE)
        
    def write(self, o):
        res = self.regex.findall(o)
        if len(res) != 0:
            print(f"\r{res[-1][0]}", end='', file=self.stdout)

class EmbeddingVectorizer:
    def __init__(self):
        """
        Embedding vectorizer
        """
        with contextlib.redirect_stdout(LookForProgress(sys.stdout)):
            fasttext.util.download_model('fr', if_exists='ignore')
        self.embedding_model = fasttext.load_model("./cc.fr.300.bin")

    def __call__(self, address):
        """
        Convert address to embedding vectors
        :param address: The address to convert
        :return: The embeddings vectors
        """
        embeddings = []
        for word in address.split():
            embeddings.append(self.embedding_model[word])
        return embeddings
     
embedding_model = EmbeddingVectorizer()

We also need a vectorizer to convert the address tag (e.g. StreeNumber, StreetName) into categorical values. So we will use a Vectorizer class that can use the embedding vectorizer and convert the address tag.

In [10]:
class Vectorizer:
    def __init__(self, dataset, embedding_model):
        self.data = dataset
        self.embedding_model = embedding_model
        self.tags_set = {
            "StreetNumber": 0,
            "StreetName": 1,
            "Unit": 2,
            "Municipality": 3,
            "Province": 4,
            "PostalCode": 5,
            "Orientation": 6,
            "GeneralDelivery": 7
        }
        
    def __len__(self):
        # for the dataloader
        return len(self.data)

    def __getitem__(self, item):
        data = self.data[item]
        address = data[0]
        address_vector = self.embedding_model(address)

        tags = data[1]
        idx_tags = self._convert_tags_to_idx(tags)

        return address_vector, idx_tags

    def _convert_tags_to_idx(self, tags):
        idx_tags = []
        for tag in tags:
            idx_tags.append(self.tags_set[tag])
        return idx_tags
    

In [11]:
train_data_vectorize = Vectorizer(train_data, embedding_model)
valid_data_vectorize = Vectorizer(valid_data, embedding_model)
test_data_vectorize = Vectorizer(test_data, embedding_model)

### DataLoader

Now, since all the addresses are not of the same size, it is impossible to batch them together since all elements of a tensor must have the same lengths. But there is a trick, padding!

The idea is simple. We add *empty* tokens at the end of each sequence up to the longest one in a batch. For the word vectors, we add vectors of 0 as padding. For the tag indices, we pad with -100s. We do so because of the [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), the accuracy metric and the [F1 metric](https://poutyne.org/metrics.html#poutyne.FBeta) all ignore targets with values of -100.

To do this padding, we use the `collate_fn` argument of the [PyTorch `DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), and on running time, that process will be done. One thing to take into account, since we pad the sequence, we need each sequence's lengths to unpad them in the forward pass. That way, we can pad and pack the sequence to minimize the training time (read [this good explanation](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch) of why we pad and pack sequences).

In [12]:
def pad_collate_fn(batch):
    """
    The collate_fn that can add padding to the sequences so all can have 
    the same length as the longest one.

    Args:
        batch (List[List, List]): The batch data, where the first element 
        of the tuple are the word idx and the second element are the target 
        label.

    Returns:
        A tuple (x, y). The element x is a tuple containing (1) a tensor of padded 
        word vectors and (2) their respective lengths of the sequences. The element 
        y is a tensor of padded tag indices. The word vectors are padded with vectors 
        of 0s and the tag indices are padded with -100s. Padding with -100 is done 
        because the cross-entropy loss, the accuracy metric and the F1 metric ignores 
        the targets with values -100.
    """

    # This gets us two lists of tensors and a list of integer. 
    # Each tensor in the first list is a sequence of word vectors.
    # Each tensor in the second list is a sequence of tag indices.
    # The list of integer consist of the lengths of the sequences in order.
    sequences_vectors, sequences_labels, lengths = zip(*[
        (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors)) 
         for (seq_vectors, labels) in sorted(batch, key=lambda x: len(x[0]), reverse=True)
    ])

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)

    padded_sequences_labels = pad_sequence(sequences_labels, batch_first=True, padding_value=-100)

    return (padded_sequences_vectors, lengths), padded_sequences_labels

In [13]:
train_loader = DataLoader(train_data_vectorize, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn, num_workers=2)
valid_loader = DataLoader(valid_data_vectorize, batch_size=batch_size, collate_fn=pad_collate_fn, num_workers=2)
test_loader = DataLoader(test_data_vectorize, batch_size=batch_size, collate_fn=pad_collate_fn, num_workers=2)

## Full Network

Since our sequences are of variable lengths and we want to be the most efficient possible by packing them, we cannot use the [PyTorch `nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class to define our model, so we define the forward pass for it to pack and unpack the sequences (again, you can read [this good explanation](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch) of why we pad and pack sequences).

In [14]:
class FullNetWork(nn.Module):
    def __init__(self, lstm_network, fully_connected_network):
        super().__init__()
        self.hidden_state = None
        
        self.lstm_network = lstm_network
        self.fully_connected_network = fully_connected_network
        
    def forward(self, padded_sequences_vectors, lengths):
        """
            Defines the computation performed at every call.
        """
        total_length = padded_sequences_vectors.shape[1]

        pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

        lstm_out, self.hidden_state = self.lstm_network(pack_padded_sequences_vectors)
        lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True, total_length=total_length)

        tag_space = self.fully_connected_network(lstm_out)
        return tag_space.transpose(-1, 1) # we need to transpose since it's a sequence

full_network = FullNetWork(lstm_network, fully_connected_network)

## Summary

So we have created an LSTM network (`lstm_network`), a fully connected network (`fully_connected_network`), those two components are used in the full network. This full network used padded, packed sequences (defined in the forward pass), so we created the `pad_collate_fn` function to process the needed work. The DataLoader will conduct that process. Finally, when we load the data, this will be done using the vectorizer, so the address will be represented using word embeddings. Also, the address components will be converted into categorical value (from 0 to 7).

# The Training Loop

Now that we have all the components for the network let's define our SGD optimizer.

In [15]:
optimizer = optim.SGD(full_network.parameters(), lr)

## Poutyne Callbacks

One nice feature of Poutyne is [callbacks](https://poutyne.org/callbacks.html). Callbacks allow doing actions during the training of the neural network. In the following example, we use three callbacks. One that saves the latest weights in a file to be able to continue the optimization at the end of training if more epochs are needed. Another one that saves the best weights according to the performance on the validation dataset. Finally, another one that saves the displayed logs into a TSV file.

In [16]:
name_of_network = "lstm_unidirectional"

callbacks = [
        # Save the latest weights to be able to continue the optimization at the end for more epochs.
        ModelCheckpoint(name_of_network + '_last_epoch.ckpt', temporary_filename='last_epoch.ckpt.tmp'),

        # Save the weights in a new file when the current model is better than all previous models.
        ModelCheckpoint(name_of_network + '_best_epoch_{epoch}.ckpt', monitor='val_acc', mode='max', save_best_only=True, restore_best=True, verbose=True, temporary_filename='best_epoch.ckpt.tmp'),

        # Save the losses and accuracies for each epoch in a TSV.
        CSVLogger(name_of_network + '_log.tsv', separator='\t'),
    ]

## Making Your own Callback

While Poutyne provides a great number of [predefined callbacks](https://poutyne.org/callbacks.html), it is sometimes useful to make your own callback.

In the following example, we want to see the effect of temperature on the optimization of our neural network. To do so, we either increase or decrease the temperature during the optimization. As one can see in the result, temperature either as no effect or has a detrimental effect on the performance of the neural network. This is so because the temperature has for effect to artificially changing the learning rates. Since we have found the right learning rate, increasing or decreasing, it shows no improvement on the results.

In [17]:
class CrossEntropyLossWithTemperature(nn.Module):
    """
    This loss module is the cross-entropy loss function
    with temperature. It divides the logits by a temperature
    value before computing the cross-entropy loss.

    Args:
        initial_temperature (float): The initial value of the temperature.
    """

    def __init__(self, initial_temperature):
        super().__init__()
        self.temperature = initial_temperature
        self.celoss = nn.CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        y_pred = y_pred / self.temperature
        return self.celoss(y_pred, y_true)


class TemperatureCallback(Callback):
    """
    This callback multiply the loss temperature with a decay before
    each batch.

    Args:
        celoss_with_temp (CrossEntropyLossWithTemperature): the loss module.
        decay (float): The value of the temperature decay.
    """
    def __init__(self, celoss_with_temp, decay):
        super().__init__()
        self.celoss_with_temp = celoss_with_temp
        self.decay = decay

    def on_train_batch_begin(self, batch, logs):
        self.celoss_with_temp.temperature *= self.decay

So our loss function will be the cross-entropy with temperature with an initial temperature of `0.1` and a temperature decay of `1.0008`.

In [18]:
loss_function = CrossEntropyLossWithTemperature(0.1)
callbacks = callbacks + [TemperatureCallback(loss_function, 1.0008)]

Now let's test our training loop for one epoch using the accuracy as the batch metric.

In [19]:
model = Model(full_network, optimizer, loss_function, batch_metrics=['accuracy'])
model.to(device)
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1, 
                    callbacks=callbacks)

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|22.88s loss: 0.113521 acc: 97.057626 val_loss: 0.068427 val_acc: 98.900570
Epoch 1: val_acc improved from -inf to 98.90057, saving file to lstm_unidirectional_best_epoch_1.ckpt
Restoring model from lstm_unidirectional_best_epoch_1.ckpt


[{'epoch': 1,
  'loss': 0.113520683106035,
  'time': 22.879249463090673,
  'acc': 97.05762597122192,
  'val_loss': 0.068426917552948,
  'val_acc': 98.90056986083984}]

## Coloring
Also, Poutyne use by default a coloring template of the training step when the package `colorama` is installed.
One could either remove the coloring (`progress_options=dict(coloring=False)`) or set a different coloring template using the fields:
`text_color`, `ratio_color`, `metric_value_color`, `time_color` and `progress_bar_color`.
If a field is not specified, the default colour will be used.

Here an example where we set the `text_color` to MAGENTA and the `ratio_color` to BLUE.

In [20]:
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1, 
                    callbacks=callbacks,
                    progress_options=dict(coloring={"text_color": "MAGENTA", "ratio_color":"BLUE"}))

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|23.10s loss: 0.108904 acc: 98.391035 val_loss: 0.182245 val_acc: 97.490286
Restoring model from lstm_unidirectional_best_epoch_1.ckpt


[{'epoch': 1,
  'loss': 0.10890439251959323,
  'time': 23.103727217996493,
  'acc': 98.39103530883789,
  'val_loss': 0.1822452466249466,
  'val_acc': 97.49028603515625}]

## Epoch metrics
It's also possible to used epoch metrics such as [`F1-score`](https://poutyne.org/metrics.html#poutyne.FBeta). You could also define your own epoch metric using the [`EpochMetric`](https://poutyne.org/metrics.html#epoch-metric-interface) interface.

In [21]:
model = Model(full_network, 
              optimizer, 
              loss_function, 
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1'])
model.to(device)
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1, 
                    callbacks=callbacks)

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|24.88s loss: 0.656512 acc: 91.617560 fscore_micro: 0.916008 val_loss: 0.852732 val_acc: 92.166623 val_fscore_micro: 0.921452
Restoring model from lstm_unidirectional_best_epoch_1.ckpt


[{'epoch': 1,
  'loss': 0.6565121836900711,
  'time': 24.882459026994184,
  'acc': 91.61756011047363,
  'fscore_micro': 0.9160076975822449,
  'val_loss': 0.8527321508407593,
  'val_acc': 92.16662268066406,
  'val_fscore_micro': 0.9214516282081604}]

Furthermore, you could also use the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) wrapper to wrap a Scikit-learn metric as an epoch metric. Below, we show how to compute the AUC ROC using the [`SKLearnMetrics`](https://poutyne.org/metrics.html#poutyne.SKLearnMetrics) class. We have to inherit the class so that the data is passed into the right format for the scikit-learn `roc_auc_score` function.

In [22]:
class FlattenSKLearnMetrics(SKLearnMetrics):
    def forward(self, y_pred, y_true):
        y_pred = y_pred.softmax(1)
        y_pred = y_pred.transpose(2, 1).flatten(0, 1)
        y_true = y_true.flatten()
        return super().forward(y_pred, y_true)

roc_epoch_metric = FlattenSKLearnMetrics(roc_auc_score, 
                                         kwargs=dict(multi_class='ovr', average='macro'))
model = Model(full_network, 
              optimizer, 
              loss_function, 
              batch_metrics=['accuracy'], 
              epoch_metrics=['f1', roc_epoch_metric])
model.to(device)
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1, 
                    callbacks=callbacks)

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|27.04s loss: 1.892510 acc: 67.241331 fscore_micro: 0.672658 roc_auc_score: 0.533377 val_loss: 1.963519 val_acc: 57.172470 val_fscore_micro: 0.572064 val_roc_auc_score: 0.572676
Restoring model from lstm_unidirectional_best_epoch_1.ckpt


[{'epoch': 1,
  'loss': 1.8925100725173951,
  'time': 27.04451349005103,
  'acc': 67.24133077850342,
  'fscore_micro': 0.672658383846283,
  'roc_auc_score': 0.5333772944692509,
  'val_loss': 1.9635190134048461,
  'val_acc': 57.17247003173828,
  'val_fscore_micro': 0.5720635652542114,
  'val_roc_auc_score': 0.5726757926701154}]

## Metric naming

It's also possible to name the metric using a tuple format `(<metric name>, metric)`. That way, it's possible to use multiple times the same metric type (i.e. having micro and macro F1-score).

In [23]:
model = Model(full_network, 
              optimizer, 
              loss_function, 
              batch_metrics=[("My accuracy name", 'accuracy')], 
              epoch_metrics=[("My metric name", 'f1')])
model.to(device)
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1)

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|25.21s loss: 2.049683 My accuracy name: 85.992988 My metric name: 0.859858 val_loss: 2.036902 val_My accuracy name: 76.003944 val_My metric name: 0.760235


[{'epoch': 1,
  'loss': 2.04968338060379,
  'time': 25.206398552982137,
  'My accuracy name': 85.99298800354003,
  'My metric name': 0.8598584532737732,
  'val_loss': 2.0369023300170896,
  'val_My accuracy name': 76.00394389648437,
  'val_My metric name': 0.7602346539497375}]

## Multi-GPUs

Finally, it's also possible to use multi-GPUs for your training either by specifying a list of devices or using the arg `"all"` to take them all.

> Obviously, you need more than one GPUs for that option.

In our case here, multi-gpus takes more time because the task is not big enough to profit from multi-gpus.

In [24]:
model = Model(full_network, 
              optimizer, 
              loss_function, 
              batch_metrics=[("My accuracy name", 'accuracy')], 
              epoch_metrics=[("My metric name", 'f1')])
model.to("all")
model.fit_generator(train_loader, 
                    valid_loader, 
                    epochs=1)

Epoch: 1/1 Step: 2500/2500 100.00% |█████████████████████████|36.12s loss: 2.020698 My accuracy name: 70.953107 My metric name: 0.709768 val_loss: 2.002923 val_My accuracy name: 63.193099 val_My metric name: 0.632155


[{'epoch': 1,
  'loss': 2.0206981167793274,
  'time': 36.123627974884585,
  'My accuracy name': 70.95310654754638,
  'My metric name': 0.7097678184509277,
  'val_loss': 2.002922854232788,
  'val_My accuracy name': 63.19309873657227,
  'val_My metric name': 0.6321554183959961}]